In [26]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as ml
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings('ignore') #pygraphviz warnings
ml.rcParams['figure.figsize'] = [32, 32]

In [3]:
rawLinks = pd.read_csv('data/9606.protein.links.v10.5.txt', delim_whitespace=True)
rawAlias = pd.read_csv('data/9606.protein.aliases.v10.5.txt', delimiter='\t')
print('Here')
print(str(datetime.datetime.now()))

Here
2019-01-16 10:19:48.721157


In [4]:
#build a map of proteinId in rawLinks to gene product names
proteinIdToAlias = {}
for val in list(zip(rawAlias.string_protein_id, rawAlias.alias)):
    if not val[0] in proteinIdToAlias: #the alias file has lots of duplicates!
        proteinIdToAlias[val[0]]=[val[1]]
    else:
        proteinIdToAlias[val[0]].append(val[1])
        
print('Here')
print(str(datetime.datetime.now()))

Here
2019-01-16 10:19:50.003406


In [5]:
#build the list of links mapped to gene product name
links = list(zip(rawLinks.protein1, rawLinks.protein2, rawLinks.combined_score))
print('Here')
print(str(datetime.datetime.now()))

Here
2019-01-16 10:19:52.814888


In [22]:
#filter links where both in and out are in the dataset
acceptableGeneList = pd.read_csv('Diff_1_3_01.txt', header=None)
#acceptableGeneList = ['ARF5','AK302958']
acceptableGeneSet = set(acceptableGeneList[0])

print(len(acceptableGeneList))
print(str(datetime.datetime.now()))

filteredLinks = []
for link in links:
    aliasesOfStart = proteinIdToAlias[link[0]]
    aliasesOfEnd = proteinIdToAlias[link[1]]
    
    startIsAcceptable = not acceptableGeneSet.isdisjoint(aliasesOfStart)
    endIsAcceptable = not acceptableGeneSet.isdisjoint(aliasesOfEnd)
    
    if startIsAcceptable and endIsAcceptable:
        filteredLinks.append(link)
        
print(len(filteredLinks))
print(str(datetime.datetime.now()))

2995
2019-01-16 10:28:10.196119
383046
2019-01-16 10:29:55.364359


In [28]:
print(str(datetime.datetime.now()))
G = nx.Graph()
G.add_weighted_edges_from(filteredLinks)
nx.nx_pydot.write_dot(G, "grid.dot")
print("Now run: sfdp -Tpng -Goverlap=false -Nshape=none grid.dot > grid.png")
print(str(datetime.datetime.now()))

2019-01-16 10:34:20.563584
Now run: sfdp -Tpng -Goverlap=false -Nshape=none grid.dot > grid.png
2019-01-16 10:37:26.224826


In [ ]:
#nx.draw(G,pos=nx.nx_pydot.graphviz_layout(G, prog='sfdp'), with_labels=True, node_shape='d')